# CICERO -- SUBER with NRMS as the model to be trained

How can we take the NRMS model and use it as input for SUBER?  By that I mean, how it provides the data in the form that the original NRMS model uses 

The NRMS  data comes out of the recommenders module itself.  After one does a pip install recommenders. 